# Spanning Tree

Aunque no sea parte del temario de grafos. Veamos cómo hacer un árbol de expansión mínima 

In [ ]:
type grafo
    vertices::Int64
    aristas::Array{Int,2}
end

In [ ]:
srand(1)
vertices=2500
aristas=Array{Int64}(vertices,vertices)
for i in 1:vertices
    aristas[i,i]=0
    for j in i+1:vertices
       aristas[i,j]=aristas[j,i]=rand(1:1000)
    end
end
instancia=grafo(vertices,aristas) #creamos un grafo denso
#println(instancia)

## Prim:

* empieza con un conjunto formado por un único vértice (el 1 por ejemplo)
* va añadiendo vértices al conjunto (uno a uno)
* el vértice que añade es uno con arista de coste mínima entre él y cualquier miembro del conjunto



In [ ]:
#Versión 1. Guardaremos el conjunto de los que pertenencen como un vector y miraremos todas las parejas una vez
function Prim_version1(instancia)
    inSpanningTree=Array{Bool}(instancia.vertices)
    inSpanningTree[1]=true
    for i in 2:instancia.vertices
        inSpanningTree[i]=false
    end
    totalCost=0
    for iteracion in 1:instancia.vertices-1 #hay que repetir el bucle vertices-1 veces
        bestCost=typemax(Int64)
        best=0
        for i in 1:instancia.vertices
            if inSpanningTree[i]==true
                for j in 1:instancia.vertices
                    if (inSpanningTree[j]==false) && (bestCost > instancia.aristas[i,j])
                        best = j
                        bestCost = instancia.aristas[i,j]
                    end
                end
            end
        end
        @assert(best != 0)
        #println("seleccionamos ",best," con costo ",bestCost)
        totalCost += bestCost
        inSpanningTree[best]=true
    end
    println("costo Total ",totalCost)
    return totalCost
end
#Versión 2. Miraremos mejor candidato para cada uno
function Prim_version2(instancia)
    #sólo para demostrar los problemas de las variables globales, sustituyan instancia.vertices por vertices
    costCandidate=Array{Int64}(instancia.vertices) 
    inSpanningTree=Array{Bool}(instancia.vertices)
    costCandidate[1]=0
    inSpanningTree[1]=true
    for i in 2:instancia.vertices #inicialización
        costCandidate[i]=instancia.aristas[i,1]
        inSpanningTree[i]=false
    end
    totalCost=0
    for iteracion in 1:instancia.vertices-1
        best=0
        for ii in 2:instancia.vertices
            if inSpanningTree[ii]==false
                best=ii
                break
            end
        end
        for i in best+1:instancia.vertices
            if inSpanningTree[i]==false && costCandidate[best]>costCandidate[i]
                best=i
            end
        end
        @assert(costCandidate[best]<typemax(Int64))
        #println("seleccionamos ",best," con costo ",costCandidate[best])
        inSpanningTree[best]=true
        totalCost += costCandidate[best]
        costCandidate[best]=typemax(Int64)
        for i in 2:instancia.vertices
            if inSpanningTree[i]==false && costCandidate[i]>instancia.aristas[i,best]
                costCandidate[i]=instancia.aristas[i,best]
            end
        end
    end
    println("costo Total ",totalCost)
    return totalCost
end

In [ ]:
@time(Prim_version1(instancia))

@time(Prim_version2(instancia))

# Ejercicio

Existen alternativas a las dos implementaciones mostradas. Yo normalmente uso la 2, excepto en grafo "sparse" que requieren implementaciones dependientes de la estructura de los datos (es más, para esos grafos casi siempre uso Kruskal)

## Kruskal:

* empieza con cada vértice formando un grupo
* va uniendo conjuntos
* el conjunto que une es el asociado a la arista de mínimo costo entre dos conjuntos diferentes





In [ ]:
#Versión 1. Guardaremos el conjunto de los que pertenencen como un vector y miraremos todas las parejas una vez
function kruskal_version1(instancia)
    grupo=Array{Int64}(instancia.vertices) 
    for i in 1:instancia.vertices
        grupo[i]=i
    end
    totalCost=0
    for iteracion in 1:instancia.vertices-1
        bestCost=typemax(Int64)
        v1=v2=0
        for i in 1:instancia.vertices
            for j in i+1:instancia.vertices
                if grupo[i]!=grupo[j] && instancia.aristas[i,j]<bestCost
                    bestCost=instancia.aristas[i,j]
                    v1=i
                    v2=j
                end
            end
        end
        @assert(v1!=0)
        totalCost+=bestCost
        grupoConservar=min(grupo[v1],grupo[v2])
        grupoEliminar=max(grupo[v1],grupo[v2])
        for i in 1:instancia.vertices
            if grupo[i]>grupoEliminar 
                grupo[i]=grupo[i]-1
            else
                if grupo[i]==grupoEliminar
                    grupo[i]=grupoConservar
                end
            end
        end
    end
    println("costo Total ",totalCost)
    return totalCost
end
    
#Versión 2. Vamos a ordenar las aristas para evitar pasar varias veces por ellas
#struct arista #utilizable desde Julia 0.6
#immutable arista
type arista
    v1::Int64
    v2::Int64
    length::Int64
end

function kruskal_version2(instancia)
    grupo=Array{Int64}(instancia.vertices) 
    ordenAristas=Array{arista}(0)
    for i in 1:instancia.vertices
        grupo[i]=i
    end
    totalCost=0
    numGrupos=instancia.vertices
    for i in 1:instancia.vertices
        for j in i+1:instancia.vertices
            push!(ordenAristas,arista(i,j,instancia.aristas[i,j]))
        end
    end
    sort!(ordenAristas,by=a->a.length) #https://rosettacode.org/wiki/Sort_an_array_of_composite_structures#Julia
    #for p in ordenAristas
    #    println(p.v1,"\t",p.v2,"\t",p.length)
    #end
    totalCost=0
    for arista in ordenAristas
        if grupo[arista.v1]!=grupo[arista.v2]
            totalCost += arista.length
            
            grupoConservar=min(grupo[arista.v1],grupo[arista.v2])
            grupoEliminar=max(grupo[arista.v1],grupo[arista.v2])
            for i in 1:instancia.vertices
                if grupo[i]>grupoEliminar 
                    grupo[i]=grupo[i]-1
                else
                    if grupo[i]==grupoEliminar
                        grupo[i]=grupoConservar
                    end
                end
            end

        end
    end
    return totalCost
end

In [ ]:
#@time(kruskal_version1(instancia))

@time(kruskal_version2(instancia))

In [ ]:
@time(kruskal_version1(instancia))